In [1]:
%pip install "pybamm[plot,cite]" -q

import pybamm

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

# Set the model parameters
param = pybamm.ParameterValues("OKane2022")
param.update({
    "SEI solvent diffusivity [m2.s-1]": 2e-21  # New value, adjustable as needed
})
var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particle
    "r_p": 30,  # Positive electrode particle
}

# Set the number of cycles
max_cycles = 500  # Only need up to the 500th cycle

# Define the experiment protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",  # Initial capacity check
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 1C until 2.5 V",  # Aging cycles
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * 500
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # Final capacity check
)

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract LLI loss (in A.h) and calculate Remaining Capacity (%)
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600  # Convert units
remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100  # Calculate remaining capacity percentage

# Output the remaining capacity at cycle 500
print(f"When the cycle number is 500, the remaining capacity is: {remaining_capacity_k:.2f}%")



Note: you may need to restart the kernel to use updated packages.


At t = 479.946 and h = 1.40182e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 239.946 and h = 1.32955e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 418.114 and h = 1.21721e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 178.114 and h = 1.23809e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 416.49 and h = 4.94051e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 176.496 and h = 4.41192e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 415.031 and h = 3.58588e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 175.031 and h = 1.10869e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 413.678 and h = 5.22305e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 173.678 and h = 7.12018e-13, the corrector convergence failed repeatedly or with |h| = hmin.
A

When the cycle number is 500, the remaining capacity is: 94.45%
